#### Libraries

In [70]:

import matplotlib.pyplot as plt
import numpy as np
import random
import json

#### Read file

In [79]:
class data_preprocessing:
    
    def __init__(self,instance_path):
        
        self.instance_path=instance_path
        
        self.info, self.flights = self.read_file(f_name=self.instance_path)
        self.flights_by_day_dict = self.flights_by_day(flight_list=self.flights)
        self.airports_by_area = self.get_airports_by_areas()
        
    def read_file(self,f_name):
        dist = []
        line_nu = -1
        with open(f_name) as infile:
            for line in infile:
                line_nu += 1
                if line_nu == 0:
                    index = int(line.split()[0]) * 2 + 1
                if line_nu >= index:
                    temp = line.split()
                    temp[2] = int(temp[2])
                    temp[3] = int(temp[3])
                    dist.append(temp)
                else:
                    dist.append(line.split())
            info = dist[0:int(dist[0][0])*2+1]
            flights = dist[int(dist[0][0])*2+1:]
        return info, flights
    
    def flights_by_day(self,flight_list):
        # Create an empty dictionary to hold flights organized by day
        flights_by_day = {}

        # Iterate over each flight in the input list
        for flight in flight_list:
            # Extract the day from the flight entry
            day = flight[2]

            # Create a flight entry without the day
            flight_without_day = flight[:2] + flight[3:]

            # Add the flight to the corresponding day in the dictionary
            if day not in flights_by_day:
                flights_by_day[day] = []
            flights_by_day[day].append(flight_without_day)

        with open("OUTPUT dictionnary", 'w') as file:
            # Serialize the dictionary to a JSON formatted string and write it to the file
            json.dump(flights_by_day, file)
            
        return flights_by_day
    
    def get_cost(self, day, from_airport, to_airport):
        flights = self.flights_by_day_dict.get(day, [])
        for flight in flights:
            if flight[0] == from_airport and flight[1] == to_airport:
                return flight[2]
        return float('inf')
    
    def get_airports_by_areas(self):
        areas = {}
        area_num = int(self.info[0][0])
        for i in range(0, area_num):
            areas[f"area_{i}"] = self.info[2+i * 2]
        return areas
    
Data_Preprocessing=data_preprocessing(instance_path="Flight connections dataset/13.in")

#### Have to test

In [ ]:
import numpy as np
import random

# MCTS Algorithm
class Node:
    def __init__(self, state, parent=None):
        self.state = state  # (current_airport, visited_areas, day, total_cost)
        self.parent = parent
        self.children = []
        self.visits = 0
        self.reward = 0
    
    def is_fully_expanded(self, areas):
        return len(self.children) == len(areas) - len(self.state[1])

    def best_child(self, c_param=1.4):
        choices_weights = [
            (child.reward / child.visits) + c_param * np.sqrt((2 * np.log(self.visits) / child.visits))
            for child in self.children
        ]
        return self.children[np.argmax(choices_weights)]

def get_possible_moves(state, areas):
    current_airport, visited_areas, day, _ = state
    next_area = [area for area in areas if area not in visited_areas][0]
    return [(airport, visited_areas + [next_area], day + 1, 0) for airport in areas[next_area]]

def rollout(state, areas, data):
    current_airport, visited_areas, day, total_cost = state
    while len(visited_areas) < len(areas):
        next_area = [area for area in areas if area not in visited_areas][0]
        next_airport = random.choice(areas[next_area])
        total_cost += data.get_cost(day + 1, current_airport, next_airport)
        current_airport = next_airport
        visited_areas.append(next_area)
        day += 1
    total_cost += data.get_cost(day + 1, current_airport, state[0])  # Return to start area
    return total_cost

def backpropagate(node, reward):
    while node:
        node.visits += 1
        node.reward += reward
        node = node.parent

def mcts(root, areas, data, iterations):
    for _ in range(iterations):
        node = root
        # Selection
        while node.is_fully_expanded(areas):
            node = node.best_child()
        
        # Expansion
        if not node.is_fully_expanded(areas):
            new_state = random.choice(get_possible_moves(node.state, areas))
            total_cost = node.state[3] + data.get_cost(node.state[2] + 1, node.state[0], new_state[0])
            new_state = (new_state[0], new_state[1], new_state[2], total_cost)
            child_node = Node(new_state, node)
            node.children.append(child_node)
            node = child_node
        
        # Simulation
        reward = rollout(node.state, areas, data)
        
        # Backpropagation
        backpropagate(node, -reward)  # Minimize cost

    return root.best_child(c_param=0)  # Return the child with the highest reward

# Example usage
data = DataPreprocessing(instance_path="Flight connections dataset/1.in")
areas = data.get_areas()

# Initialize the root state
start_airport = data.info[1][1]  # Starting airport
start_state = (start_airport, [], 0, 0)
root = Node(start_state)

# Perform MCTS
best_node = mcts(root, areas, data, 1000)

# Print the best state found
print("Best route found:")
print(best_node.state)